### In this file I will try to further develop the model and use the inspiration from the previous bidding files. The goal is to end up with a deep reinforcement learning model that can handle the bidding process and be able to make bids without knowing the prices. 

In [1]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, weather, timeframes
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict, Counter
import random
import sklearn

In [2]:
tf = timeframes.one_week

In [3]:
L, M, F, H, freq_data, power_meter_dict, consumption_data = utils.get_all_sets(timeframe= tf, areas = ["NO5"])

In [4]:
L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m = utils.get_parameters(L = L, M = M, H = H)

In [5]:
Ir_hlm, Ia_hlm, Va_hm = utils.get_income_dictionaries(H=H, L = L, M = M, freq_data= freq_data, Fu_h_l= Fu_h_l, Fd_h_l= Fd_h_l, P_h_m= P_h_m, Vp_h_m= Vp_h_m, F = F, markets_dict = {market.name : market for market in M}, timeframe = tf, areas = ["NO5"])

/Users/sandermeland/Documents/UIB/Master/master-kode/code_map/utils.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)
/Users/sandermeland/Documents/UIB/Master/master-kode/code_map/utils.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)


In [6]:
compatible_dict = utils.get_compatibility_dict(L = L ,M = M, index = False)

In [7]:
len(L)

960

In [276]:
total_up_flex = np.sum(Fu_h_l) # total available flex volume up
total_down_flex = np.sum(Fd_h_l) # total available flex volume down
total_response_time = np.sum(R_h_l) # total response time
#total_flex = total_up_flex + total_down_flex
average_response_time = total_response_time/ (len(H)*len(L))
hourly_flex_up = total_up_flex/len(H)
hourly_flex_down = total_down_flex/len(H)

print(f"Total up flex volume: {total_up_flex} MW")
print(f"Total down flex volume: {total_down_flex} MW")
print(f"Average flex volume pr hour up: {hourly_flex_up} MWh")
print(f"Average flex volume pr hour down: {hourly_flex_down} MWh")
print(f"Average response time: {average_response_time} seconds")

Total up flex volume: 532.630979 MW
Total down flex volume: 556.472583 MW
Average flex volume pr hour up: 3.170422494047619 MWh
Average flex volume pr hour down: 3.3123368035714287 MWh
Average response time: 144.45269926517432 seconds


In [8]:
def random_arg_max(possible_actions):
    imax = 0
    xmax = possible_actions[imax]  # Current maximum
    nmax = 1  # Number of maximum values at the moment
    for i in range(1, len(possible_actions)):
        if possible_actions[i] == xmax:
            nmax += 1
            if nmax * random.random() < 1.0:
                imax = i
        elif possible_actions[i] > xmax:
            nmax = 1  # Reset count since a new maximum is found
            imax = i
            xmax = possible_actions[i]  # Update the new maximum
    return imax


In [9]:
def greedy_action(possible_actions : list, epsilon : float ):
    """returns the index of the greedy action

    Args:
        possible_actions (lsit): list of the possible actions
        epsilon (float): float number between 0 and 1, often close to 0

    Returns:
        int: index of the greedy action
    """
    if random.random() <= (1- epsilon): # pick greedy
        return random_arg_max(possible_actions)
    else:
        return random.randint(0, len(possible_actions)-1) # random


In [10]:
def get_expected_prices_and_volumes_dict(bid_timeframe, markets):
    """ function to calculate the expected prices for each direction and area in the bid_timeframe

    Args:
        bid_timeframe (_type_): _description_
        markets (_type_): _description_

    Returns:
        _type_: _description_
    """
    expected_prices = {}
    expected_volumes = {}
    for directions in ["up", "down", "both"]:
        #for area in ["NO1", "NO2", "NO3", "NO4", "NO5"]:
        area = "NO5"
        for hour in bid_timeframe:
            if np.isnan(np.mean([market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in markets if market.area == area and market.direction == directions])):
                print("nan")
                print([market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in markets if market.area == area and market.direction == directions])
                print(f"directions: {directions}, area: {area}, hour: {hour}")
            expected_prices[(directions, area, hour)] = np.mean([market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in markets if market.area == area and market.direction == directions])
            if np.isnan(np.mean([market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1] for market in markets if market.area == area and market.direction == directions])):
                print("nan")
                print([market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1] for market in markets if market.area == area and market.direction == directions])
                print(f"directions: {directions}, area: {area}, hour: {hour}")
            expected_volumes[(directions, area, hour)] = np.mean([market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1] for market in markets if market.area == area and market.direction == directions])
    return expected_prices, expected_volumes


In [11]:
sup_market_names = ["FCR", "aFRR"]
markets = [market for market in M if sup_market_names[0] in market.name  or sup_market_names[1] in market.name]
exp_price_dict, exp_vol_dict = get_expected_prices_and_volumes_dict(bid_timeframe= H, markets= markets)

In [12]:
exp_price_dict[("up", "NO5", H[4])]

3.8666666666666667

In [13]:
exp_vol_dict[("up", "NO5", H[4])]

2.0

In [14]:
def normalize_dict_vals(dict : dict, norm_method :str = "min_max" ) -> dict:
    """Function to normalize the values of a dictionary

    Args:
        dict (dict): [description]
        norm_method (str): Normalization method; can choose between min-max normalization or z-score normalization. Defaults to "min_max".

    Returns:
        dict: the same dictionary as input, but with normalized values
    """
    if norm_method == "min_max":
        values = list(dict.values())
        min_value = min(values)
        max_value = max(values)

        # Normalize and update the dictionary
        normalized_dict = {k: (v - min_value) / (max_value - min_value) for k, v in dict.items()}
    else:
        import statistics
        mean = statistics.mean(values)
        std = statistics.stdev(values)
        # Normalize and update the dictionary
        normalized_dict = {k: (v - mean) / std for k, v in dict.items()}

    return normalized_dict
 

In [15]:
norm_exp_price_dict = normalize_dict_vals(exp_price_dict, norm_method= "min_max")
norm_exp_vol_dict = normalize_dict_vals(exp_vol_dict, norm_method= "min_max")

In [16]:
norm_exp_price_dict.values()

dict_values([0.11697205256339072, 0.042939107903016846, 0.040347954839903756, 0.040347954839903756, 0.042939107903016846, 0.1254858411993337, 0.12178419396631501, 0.12178419396631501, 0.12415324819544697, 0.1110494169905608, 0.11674995372940959, 0.12215435868961688, 0.11845271145659818, 0.11845271145659818, 0.11845271145659818, 0.11141958171386267, 0.1110494169905608, 0.10364612252452342, 0.10697760503424024, 0.10364612252452342, 0.10379418841384415, 0.10697760503424024, 0.10379418841384415, 0.1110494169905608, 0.11093836757357024, 0.03668332407921525, 0.03668332407921525, 0.03668332407921525, 0.03668332407921525, 0.11686100314640015, 0.11071626873958913, 0.10216546363131596, 0.09106052193225987, 0.08883953359244864, 0.09624282805848602, 0.09624282805848602, 0.09624282805848602, 0.09624282805848602, 0.09624282805848602, 0.09624282805848602, 0.09624282805848602, 0.09624282805848602, 0.09624282805848602, 0.09624282805848602, 0.09846381639829725, 0.08735887469924117, 0.09106052193225987, 

#### Will have to determine which type of network to use, and how to implement it. May look into the INF265 tasks for inspiration. Deep Feedforward Network (Multilayer Perceptron) is interesting. May look in to attention mechanisms and how to implement them. Also actor-critic architectures is interesting. May end up in a hybrid model.

In [17]:
def get_possible_dates(date : pd.Timestamp):
    """ Function to get the possible dates for placing a bid given the current date

    Args:
        date (pd.Timestamp): the current date

    Returns:
        (pd.date_range, str): the possible dates for placing a bid and for which market
    """
    if date.hour == 17: # FCR D-2
        return (pd.date_range(date + timedelta(days=1) + timedelta(hours=7), date + timedelta(days = 2) + timedelta(hours = 6), freq='H', tz = "Europe/Oslo"), "D_2")
    elif date.hour == 7: # aFRR
        return (pd.date_range(date + timedelta(hours = 17), date + timedelta(days = 1) + timedelta(hours = 16), freq='H', tz = "Europe/Oslo"), "aFRR")
    elif date.hour == 18: # FCR D-1
        return (pd.date_range(date + timedelta(hours=6), date + timedelta(days = 1) + timedelta(hours = 5), freq='H', tz = "Europe/Oslo"), "D_1")
    else:
        return ([], "No bids")

In [134]:
def get_feasible_portfolio_for_market(possible_assets : [new_meters.PowerMeter], market : final_markets.ReserveMarket, hour : pd.Timestamp):
    """This function will return one feasible combination quickly.
        DISCLAIMER : This function doesnt take procured volume in the given market in to account.

    Args:
        possible_assets ([new_meters.PowerMeter]): list of possible assets for the given market and given hour
        market (final_markets.ReserveMarket): the market for which the portfolio is to be found
        hour (pd.Timestamp): the hour for which the portfolio is to be found

    Returns:
        list(new_meters.PowerMeter): the feasible portfolio that will be used for the given market in the given hour
    """
   
    # Fetch volumes for each asset at the given hour
    feasible_assets = [asset for asset in possible_assets if asset in compatible_dict[market]] # exclude assets that are not compatible with the given market
    # Fetch volumes for each asset at the given hour
    asset_volumes = []
    for asset in feasible_assets:
        if market.direction == "both":
            if asset.direction == "both":
                vol = min(asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0], asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0])
                asset_volumes.append((asset, vol))
            else:
                vol = asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if asset.direction == "up" else asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0]
                asset_volumes.append((asset, vol))
        else:
            vol = asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if market.direction == "up" else asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0]
            asset_volumes.append((asset, vol))
            

    # Sort assets by volume in descending order
    asset_volumes.sort(key=lambda x: x[1], reverse=True)

    # Find a feasible combination
    feasible_combination = []
    total_volume = 0
    for asset, volume in asset_volumes:
        feasible_combination.append(asset)
        total_volume += volume
        if total_volume >= market.min_volume:
            break
    
    if total_volume < market.min_volume:
        return [], 0

    return feasible_combination, total_volume



Idea for new way to define flex volume for each asset when not knowing the actual consumption for given hour. Due to the strong patterns in the consumption i can find the average consumption for each load for every hour of day and day of week as well as seasonal variations. Then i can find some sort of variance as well to define the flex volume as a range. This will be done for each load. Should ask Trond to see if this is a good idea and get some help from him. 

To do this I need a bigger dataset of the consumption. As per now i only have consumption data for june. Maybe I have to do this in databricks. Should talk to Trond about this. 

In [135]:
def get_n_portfolios_for_market(possible_assets : [new_meters.PowerMeter], market : final_markets.ReserveMarket, hour : pd.Timestamp, top_n=100, iterations=100):
    """function to find a diverse and large subset of feasible combinations without the computational overhead of checking all possible combinations.
        DISCLAIMER : This function doesnt take procured volume in the given market in to account.

    Args:
        possible_assets ([new_meters.PowerMeter]): list of possible assets for the given market and given hour
        market (final_markets.ReserveMarket): the market for which the portfolio is to be found
        hour (pd.Timestamp): the hour for which the portfolio is to be found
        top_n (int, optional): controls the number of top assets to consider. Defaults to 100.
        iterations (int, optional):  determines how many different combinations to try and generate. Defaults to 100.

    Returns:
        feasible_combinations: dict of top_n feasible combinations of assets that can be bid to the given market in the given hour where the values holds the portfolio's aggregated volume
    """
    feasible_assets = [asset for asset in possible_assets if asset in compatible_dict[market]] # exclude assets that are not compatible with the given market
    # Fetch volumes for each asset at the given hour
    
    asset_volumes = []
    for asset in feasible_assets:
        if market.direction == "both":
            if asset.direction == "both":
                vol = min(asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0], asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0])
                asset_volumes.append((asset, vol))
            else:
                vol = asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if asset.direction == "up" else asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0]
                asset_volumes.append((asset, vol))
        else:
            vol = asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if market.direction == "up" else asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0]
            asset_volumes.append((asset, vol))
            

    # Sort assets by volume in descending order
    asset_volumes.sort(key=lambda x: x[1], reverse=True)

    # Select top N assets
    top_assets = asset_volumes[:top_n]
    
    #feasible_combinations = set()
    feasible_combinations = {}
    if len(top_assets) > 0:
        for _ in range(iterations):
            # Randomly sample a smaller subset from top assets
            num_assets_to_sample = min(len(top_assets), random.randint(1, top_n))

            sampled_assets = random.sample(top_assets, k=num_assets_to_sample)

            # Greedy addition to meet minimum volume
            combination, total_volume = [], 0
            for asset, volume in sampled_assets:
                combination.append(asset)
                total_volume += volume
                if total_volume >= market.min_volume:
                    #feasible_combinations.add((tuple(combination), total_volume))
                    feasible_combinations[tuple(combination)] = total_volume
                    break
    
    return feasible_combinations


In [255]:
def get_max_portfolio_for_market(possible_assets : [new_meters.PowerMeter], market : final_markets.ReserveMarket, hour : pd.Timestamp):
    """ Heuristic to bid all feasible assets in to the market in given hour. Equivalent to greedy approach
        DISCLAIMER : This function doesnt take procured volume in the given market in to account.
    Args:
        possible_assets ([new_meters.PowerMeter]): list of possible assets for the given market and given hour
        market (final_markets.ReserveMarket): the market for which the portfolio is to be found
        hour (pd.Timestamp): the hour for which the portfolio is to be found

    Returns:
        feasible_assets ([new_meters.PowerMeter]): list of the assets that will be bid in to the given market in the given hour
        total_volume (float): the total volume of the feasible assets for the given hour
    """
    feasible_assets = [asset for asset in possible_assets if asset in compatible_dict[market]] # exclude assets that are not compatible with the given market
    # Fetch volumes for each asset at the given hour
    
    asset_volumes = []
    for asset in feasible_assets:
        if market.direction == "both":
            if asset.direction == "both":
                vol = min(asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0], asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0])
                asset_volumes.append(vol)
            else:
                vol = asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if asset.direction == "up" else asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0]
                asset_volumes.append(vol)
        else:
            vol = asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if market.direction == "up" else asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0]
            asset_volumes.append(vol)
        
    if sum(asset_volumes) >= market.min_volume:
        return feasible_assets, sum(asset_volumes)
    else:
        return [], 0

### The get_portfolio_for_market function where I try to find each and every combination of assets for a given market in a given hour is useless even though I only have NO5 assets and markets. The function is way to computationally expensive. I will have to find a way to make it more efficient or find another way of defining the action space

### It will be important to notice that the bidding deadlines are not the same for all markets. This will have to be taken into account when training the model. I will have to make some sort of iteration process that will make the model learn the bidding deadlines for each market and how this affects the bidding process. I must take in to account that the bids is not necessarily being accepted and i will have to make a method where the model will recieve a signal if the bid is accepted or not before it makes the next bid. 

### Think it will be a good idea to use RL for choosing which markets that will be bid in to and then using some sort of heuristic to determine which of the assets that will be bid in to the chosen market(s).

In [256]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [257]:
weather_data = weather.get_weather_data(tf= tf, areas = ["NO5"])

In [258]:
def normalize_weather_data(weather_data, scaler = sklearn.preprocessing.MinMaxScaler()):
    data = weather_data.copy()
    data["precipitation"] = scaler.fit_transform(data[["precipitation"]])
    data["air_temp"] = scaler.fit_transform(data[["air_temp"]])
    return data


In [259]:
norm_w_df = normalize_weather_data(weather_data= weather_data)

In [260]:
norm_w_df

,Time (Local),precipitation,air_temp,area
1,2023-06-19 00:00:00+02:00,0.0,0.645833,NO5
7,2023-06-19 01:00:00+02:00,0.0,0.611111,NO5
12,2023-06-19 02:00:00+02:00,0.0,0.541667,NO5
15,2023-06-19 03:00:00+02:00,0.0,0.520833,NO5
21,2023-06-19 04:00:00+02:00,0.0,0.520833,NO5
...,...,...,...,...
817,2023-06-25 19:00:00+02:00,0.0,0.861111,NO5
820,2023-06-25 20:00:00+02:00,0.0,0.854167,NO5
829,2023-06-25 21:00:00+02:00,0.0,0.736111,NO5
830,2023-06-25 22:00:00+02:00,0.0,0.576389,NO5


In [261]:
spot_path = "../master-data/spot_data/spot_june_23.csv"

norm_da_df = final_markets.preprocess_spot_data(pd.read_csv(spot_path), year = tf.year, start_month = tf.start_month, end_month = tf.end_month, start_day = tf.start_day, end_day = tf.end_day, start_hour = tf.start_hour, end_hour = tf.end_hour, area = "NO5", normalize= True)


In [262]:
norm_da_df.head()

,delivery_area,currency,Time(Local),product,settlement
0,NO5,EUR,2023-06-19 00:00:00+02:00,SPOT,0.528142
1,NO5,EUR,2023-06-19 01:00:00+02:00,SPOT,0.527085
2,NO5,EUR,2023-06-19 02:00:00+02:00,SPOT,0.523229
3,NO5,EUR,2023-06-19 03:00:00+02:00,SPOT,0.520804
4,NO5,EUR,2023-06-19 04:00:00+02:00,SPOT,0.519622


In [263]:
precipitation = norm_w_df["precipitation"].loc[norm_w_df["Time (Local)"] == H[4]]
temperature = norm_w_df["air_temp"].loc[norm_w_df["Time (Local)"] == H[4]]
da_price = norm_da_df["settlement"].loc[norm_da_df["Time(Local)"] == H[4]]

print(precipitation)
print(temperature)
print(da_price)

# only print the value
print(precipitation.values[0])
print(temperature.values[0])
print(da_price.values[0])

21    0.0
Name: precipitation, dtype: float64
21    0.520833
Name: air_temp, dtype: float64
4    0.519622
Name: settlement, dtype: float64
0.0
0.5208333333333334
0.5196218670315318


In [264]:
def get_features(bid_hour : pd.Timestamp, available_assets : [new_meters.PowerMeter], market : final_markets.ReserveMarket):
    """ Function to get the features for the given hour and market. It is important to use features that will help the model learn which actions to take 
        and update the weights correctly for the given state.
        The features that possibly can be used here are the following:
        - day of week
        - hour of day
        - number of possible assets
        - Day Ahead (DA) price
        - Weather forecast
        - Market historical prices
        - Market historical volumes
        - frequency data (historical)
        
    Args:
        available_assets (new_meters.PowerMeter]): _description_
        hour (pd.Timestamp): _description_
        market (final_markets.ReserveMarket): _description_

    Returns:
        _type_: _description_
    """
    day_of_week = bid_hour.weekday()
    hour_of_day = bid_hour.hour
    expected_price = norm_exp_price_dict[(market.direction, market.area, bid_hour)]
    expected_volume = norm_exp_vol_dict[(market.direction, market.area, bid_hour)]
    precipitation = norm_w_df["precipitation"].loc[norm_w_df["Time (Local)"] == bid_hour]
    temperature = norm_w_df["air_temp"].loc[norm_w_df["Time (Local)"] == bid_hour]
    da_price = norm_da_df["settlement"].loc[norm_da_df["Time(Local)"] == bid_hour]
    
    
    return [day_of_week/7, hour_of_day/24, expected_price, expected_volume, len(available_assets)/len(L), precipitation.values[0], temperature.values[0], da_price.values[0]]

In [265]:
def get_income_for_portfolio(volume : float, market : final_markets.ReserveMarket, hour : pd.Timestamp):
    """function to calculate the income for a given portfolio

    Args:
        portfolio ([new_meters.PowerMeter]): the portfolio for which the income is to be calculated
        market (final_markets.ReserveMarket): the market for which the income is to be calculated
        hour (pd.Timestamp): the hour for which the income is to be calculated

    Returns:
        float: the income for the given portfolio
    """
    
    return volume * market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1]


In [266]:
def make_bid(market : final_markets.ReserveMarket, hour : pd.Timestamp, action : int, possible_assets : [new_meters.PowerMeter]):
    """Function to make a bid to a given market in a given hour

    Args:
        market (final_markets.ReserveMarket): the market to bid to
        hour (pd.Timestamp): the hour to bid to
        action (int): the index of the portfolio to bid
        possible_assets ([new_meters.PowerMeter]): list of possible assets for the given market and given hour

    Returns:
        portfolio (list(new_meters.PowerMeter)): the portfolio that was bid
        income (float): the income for the portfolio in the given market at the given hour
    """
    if action == 0:
        return [], 0, 0
    elif action == 1:
        portfolio_dict = get_n_portfolios_for_market(possible_assets = possible_assets, market = market, hour = hour, top_n=100, iterations=100)
        portfolio = list(portfolio_dict.keys())[0] if len(portfolio_dict.keys()) > 0 else []
        volume = portfolio_dict[portfolio] if len(portfolio) > 0 else 0
        return portfolio, get_income_for_portfolio(volume, market, hour), volume
    else:
        #portfolio, volume = get_feasible_portfolio_for_market(possible_assets = possible_assets, market = market, hour = hour)
        portfolio, volume = get_max_portfolio_for_market(possible_assets = possible_assets, market = market, hour = hour)
        return portfolio, get_income_for_portfolio(volume, market, hour), volume

In [309]:
def train_model(epsilon, alpha, weights, num_episodes, L, M, H):
    """ Training function to learn how to bid in to the markets due to the bidding constraints and the asset constraints as well as optimizing the weights. 
    The function will learn how to bid in to the markets by updating the weights due to the features. 
    

    Args:
        epsilon (float): float number between 0 and 1, often close to 0
        alpha (float): float number between 0 and 1, also known as the learning rate
        num_episodes (int): number of episodes to be ran
        L ([new_meters.PowerMeter]): list of PowerMeter objects
        M ([final_markets.Reservemarket]): list of ReserveMarket objects
        H ([pd.Timestamp]): list of timestamps

    Returns:
        bids (dict): dictionary that holds control over the final bids for each market and each hour. The keys are tuples of the market name and the hour and the values are the the assets that were bid for the given market and hour. The values are tuples including the list of assets and the aggregated volume.
        revenues (dict): dictionary that holds the revenue for each episode
        available_assets (dict): dictionary that holds the available assets for each hour
        weights (np.array): the weights that were learned
        action_list (list): list of all actions that were taken
        asset_bids (list): list that holds the bids for each episode. The indexes are the episode number and the values are dataframes that holds the bids for each market and each hour for each episode.
    """
    revenues = {}
    bid_timeframe = H[24:] # the hours where bids can be placed in
    place_bid_hours = [hour for hour in H[:-48] if hour.hour == 7 or hour.hour == 17 or hour.hour == 18] # the hours where bids can be placed from
    bids = {}
    asset_bids = []
    sup_market_names = ["FCR", "aFRR"]
    markets = [market for market in M if sup_market_names[0] in market.name  or sup_market_names[1] in market.name]
    #market_names = [market.name for market in markets]
    # will only use FCR-N, FCR-D and aFRR. The FCR markets are both D-1 and D-2 and the aFRR market is D-1 but it is both up and down
    
    available_assets = {hour: L.copy() for hour in bid_timeframe} 
    #bids = {(market.name, hour): [] for hour in bid_timeframe for market in markets}
    n_actions = 3
    "For each hour, an action should be to either bid in every feasible asset, to bid the minimum volume or to bid nothing"
   
    (possible_hours, market_name) = get_possible_dates(place_bid_hours[0]) # the market_name is not the full name of the market, but rather a substring of the full name. Since FCR-D and FCR-N has the same deadlines there will be two markets with the same substring
    
    possible_markets = [m for m in markets if market_name in m.name] # will either be one or two markets
    #indexes = [market_names.index(m.name) for m in possible_markets] # the indexes for each market. This is used to slice the Q-table so it is not necessary in this case
    features = get_features(bid_hour = possible_hours[0], available_assets= available_assets[possible_hours[0]], market = possible_markets[0])
    
    action_0 = random.randint(0, n_actions-1)
    action_list = [] # list to keep track of the actions taken.
    
    # extract hours from H where hour == 7, 17, 18
    epsilon_decay = epsilon/num_episodes
    
    for episode_n in range(num_episodes):
        if episode_n > 0:
            epsilon -= epsilon_decay
            alpha -= 0.001
        revenue = 0
        episode_bid_df = pd.DataFrame(columns= ["Market", "Hour", "Asset Count", "Total Flex Volume"])
        episode_bid_dict = {}
        available_assets = {hour: L.copy() for hour in bid_timeframe}
        for place_hour in place_bid_hours:
            (possible_hours, market_name) = get_possible_dates(place_hour)
            if len(possible_hours) != 24:
                print(f"No bids for {place_hour}")
                print(f"possible_hours: {len(possible_hours)}")
                break
            possible_markets = [m for m in markets if market_name in m.name] # because i am now using only no5 markets, this list should be of length 1 or 2.
            
            for current_market in possible_markets:
                #print(f"current_market : {current_market.name}")
                for bid_hour in possible_hours:
                    
                    chosen_portfolio, reward, flex_vol = make_bid(current_market, bid_hour, action_0, available_assets[bid_hour]) # get portfolio and reward for a bid - will have to figure out a better way to define reward
                    available_assets[bid_hour] = [asset for asset in available_assets[bid_hour] if asset not in chosen_portfolio] # remove the assets that were bid from the available assets
                    if len(chosen_portfolio) > 0:
                        episode_bid_df.loc[len(episode_bid_df)] = [current_market.name, bid_hour, len(chosen_portfolio), flex_vol] # add the bid to the episode_bid_df
                    episode_bid_dict[(current_market.name, bid_hour)] = (chosen_portfolio, flex_vol) # add the bid to the bids dictionary
                    
                    new_features = get_features(bid_hour = bid_hour, available_assets= available_assets[bid_hour], market = current_market) # update the features
                    
                    possible_actions = [np.dot(weights[action], new_features) for action in range(n_actions)] 
                    
                    new_action = greedy_action(possible_actions, epsilon)
                    
                    action_list.append(new_action)
                    
                    delta = reward + np.dot(weights[new_action], new_features) - np.dot(weights[action_0], features)
                    
                    #weights = weights + alpha * delta * np.array(features + [action_0])
                    weights[action_0] = weights[action_0] + alpha * delta * features[action_0] # * features???
                    
                    features, action_0 = new_features, new_action
                    revenue += reward
                    
        bids[episode_n] = episode_bid_df.sort_values(by = ["Hour"])
        revenues[episode_n] = revenue
        asset_bids.append(episode_bid_dict)
    return bids, revenues, available_assets, weights, action_list, asset_bids

In [310]:
def initialize_weights(n_features :int , n_actions : int):
    return np.array([np.zeros((n_features)) for _ in range(n_actions)])

In [327]:
bids, revenues, available_assets, weights, action_list, asset_bids = train_model(epsilon = 0.4, alpha = 0.4, weights = initialize_weights(8, 3), num_episodes = 100, L = L, M = M, H = H)

In [202]:
[m.name for m in M[2:8]]

['FCR_D_D_1_NO5',
 'FCR_D_D_2_NO5',
 'FCR_N_D_1_NO5',
 'FCR_N_D_2_NO5',
 'aFRR up_NO5',
 'aFRR down_NO5']

In [208]:
compatible_list = utils.get_compatibility_dict(L = L ,M = M)

In [209]:
mod, x, y, w, d = utils.run_optimization_model(L = L, M = M, F = F, H = H, Fu_h_l = Fu_h_l, Fd_h_l = Fd_h_l, R_h_l = R_h_l, Vp_h_m = Vp_h_m, Vm_m = Vm_m, R_m = R_m, Ir_hlm = Ir_hlm, Ia_hlm = Ia_hlm, Va_hm = Va_hm, compatible_list= compatible_list, log_filename= "week_mod_no5.log", model_name= "week_mod_no5")

Set parameter LogFile to value "week_mod_no5.log"
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threadsThread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4687200 rows, 2262624 columns and 14380786 nonzerosOptimize a model with 4687200 rows, 2262624 columns and 14380786 nonzeros
Model fingerprint: 0x8c74d6e7Model fingerprint: 0x8c74d6e7
Model has 82782 quadratic objective termsModel has 82782 quadratic objective terms
Model has 2688 quadratic constraintsModel has 2688 quadratic constraints
Variable types: 0 continuous, 2262624 integer (2262624 binary)Variable types: 0 continuous, 2262624 integer (2262624 binary)
Coefficient statistics:Coefficient statistics:
Matrix range     [1e-06, 1e+03]  Matrix range     [1e-06,

In [225]:
def get_market_count_dict(x : dict, H : [pd.Timestamp], L : [new_meters.PowerMeter], M : [final_markets.ReserveMarket], dominant_directions):
    """ function to get a dictionary of the results of the optimization problem.
        The solution is represented as a dataframe for each hour which tells how many assets and how much flex volume is connected to each market for each hour.

    Args:
        x (dict): dictionary of the binary variable which tells if an asset is connected to a market
        L (list(PowerMeter)): list of powermeter objects with the data for each meter within the timeframe
        M (list(ReserveMarket)): list of reservemarket objects with the data for each market within the timeframe
        H (list(pd.TimeStamp)): list of hourly timestamps within the timeframe
        dominant_directions (list(str)): list of the dominant direction for each hour

    Returns:
        dict: the solution of the optimization problem
    """
    data = []

    for h, hour in enumerate(H):
        for l, load in enumerate(L):
            for m, market in enumerate(M):
                if x[h, l, m].X > 0.5:
                    # Calculate flex volume for this asset, market, and hour
        
                    data.append([hour, load.meter_id, market.name])

    df = pd.DataFrame(data, columns=["Hour", "Asset Meter ID", "Market"])
    market_names = [m.name for m in M]
    market_count_dict = {}
    for h, hour in enumerate(H):
        hour_df = df.loc[(df["Hour"] == hour)]
        # Aggregate data by market and hour, counting assets and summing flex volumes
        market_count = hour_df.groupby(["Market", "Hour"]).agg({"Asset Meter ID": "count"}).reset_index().rename(columns={"Asset Meter ID": "Asset Count"})
        flex_volumes = []
        for market_name in market_count["Market"]:
            m = market_names.index(market_name)
            market = M[m]
            if market.direction == "up":
                total_flex_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
            elif market.direction == "down":
                total_flex_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
            else:
                total_flex_volume = 0
                for load in L:
                    if load.direction == "up":
                        total_flex_volume += x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] 
                    elif load.direction == "down":
                        total_flex_volume += x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0]
                    else:
                        total_flex_volume += min(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0], x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0])
                
            flex_volumes.append(total_flex_volume)
        market_count["Total Flex Volume [MWh]"] = flex_volumes
        market_count_dict[hour] = market_count
    return market_count_dict

In [210]:
dominant_directions = [utils.get_dominant_direction(freq_data, hour) for hour in H]

In [217]:
market_count_dict = get_market_count_dict(x = x, L = L, M = M, H = H, dominant_directions= dominant_directions)

In [283]:
combined_df = pd.concat(market_count_dict, ignore_index=True)

In [284]:
display(combined_df)

,Market,Hour,Asset Count,Total Flex Volume
0,aFRR up_NO5,2023-06-19 00:00:00+02:00,580,3.351549
1,aFRR up_NO5,2023-06-19 05:00:00+02:00,531,2.706115
2,aFRR up_NO5,2023-06-19 06:00:00+02:00,529,2.383027
3,aFRR up_NO5,2023-06-19 07:00:00+02:00,543,2.800745
4,aFRR up_NO5,2023-06-19 08:00:00+02:00,575,2.930003
...,...,...,...,...
223,FCR_N_D_1_NO5,2023-06-25 20:00:00+02:00,308,1.337819
224,aFRR down_NO5,2023-06-25 20:00:00+02:00,257,2.908685
225,aFRR down_NO5,2023-06-25 21:00:00+02:00,529,4.233296
226,aFRR down_NO5,2023-06-25 22:00:00+02:00,528,4.190777


In [334]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

def make_plots_for_sol(dict_of_dfs : dict, sol_name : str = "Optimization", rl_df : pd.DataFrame = None, y_axis : str = "Asset Count"):
    """ function to make plot of the results recieved from the wanted model. This function can either plot how many assets that were bid in each market for each hour or the total flex volume that was bid in each market for each hour.

    Args:
        dict_of_dfs (dict): for the optimization model, the results are stored in a dict. The keys are the hours and the values are dataframes that holds the results for each hour.
        sol_name (str, optional): Either "Optimization" or "RL. Defaults to "Optimization".
        rl_df (pd.DataFrame, optional): The results for the RL df are stored in a dataframe and this should be given here if the results from the RL model is wanted. The RL model returns a list of dataframes, and that list with the wanted index (episode) should be given. Defaults to None.
        y_axis (str, optional): Choose which y-axis value that is wanted. Can be either "Asset Count" or "Total Flex Volume". Defaults to "Asset Count".
    """
    if sol_name == "Optimization":
        combined_df = pd.concat(dict_of_dfs, ignore_index=True)
        combined_df = combined_df.loc[combined_df["Hour"]>= pd.Timestamp(2023, 6, 20, 0, 0, 0, tz = "Europe/Oslo")]
    else:
        combined_df = rl_df.copy()
    # Create a figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    # Iterate over each market and add a scatter trace for Asset Count and Total Flex Volume
    for market in combined_df['Market'].unique():
        market_data = combined_df[combined_df['Market'] == market]
        if y_axis == "Asset Count":
            # Scatter for Asset Count
            fig.add_trace(
                go.Scatter(x=market_data['Hour'], y=market_data['Asset Count'], 
                        mode='markers', name=f"Asset Count - {market}"),
                secondary_y=False,
            )
        elif y_axis == "Total Flex Volume":
            # Scatter for Total Flex Volume
            fig.add_trace(
                go.Scatter(x=market_data['Hour'], y=market_data['Total Flex Volume'], 
                        mode='markers', name=f"Total Flex Volume - {market}"),
                secondary_y=True,
            )

    # Add figure title and layout
    fig.update_layout(title_text=f"{y_axis} by Market and Hour from the {sol_name} model")

    # Set x-axis title
    fig.update_xaxes(title_text="Hours")

    # Set y-axes titles
    fig.update_yaxes(title_text="Asset Count", secondary_y=False)
    fig.update_yaxes(title_text="Total Flex Volume", secondary_y=True)

    # Show the figure
    fig.show()

In [335]:
make_plots_for_sol(market_count_dict, y_axis = "Asset Count")

In [336]:
make_plots_for_sol(market_count_dict, sol_name= "RL", rl_df= bids[98])

In [338]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= [i for i in range(len(revenues.keys()))], y= [i for i in revenues.values()], mode= "markers"))
# add title and axis labels
fig.update_layout(title_text="Revenue per episode for RL model", xaxis_title="Episode", yaxis_title="Revenue")
# add a regression line for the revenue values for each episode.
fig.add_trace(go.Scatter(x= [i for i in range(len(revenues.keys()))], y= [i for i in revenues.values()], mode= "lines"))
fig.show()

In [349]:
weights

[array([437.51433855, 437.51433855, 437.51433855, 437.51433855,
        437.51433855, 437.51433855, 437.51433855, 437.51433855]),
 array([444.82379809, 444.82379809, 444.82379809, 444.82379809,
        444.82379809, 444.82379809, 444.82379809, 444.82379809]),
 array([542.93879871, 542.93879871, 542.93879871, 542.93879871,
        542.93879871, 542.93879871, 542.93879871, 542.93879871])]

In [346]:
def feasibility_check_for_rl_model(bids : dict, chosen_assets : list):
    """Function to check if the bids that were made are feasible. Especially want to check the flex_volume for each asset and see if it is enough to cover the bids that were made

    Args:
        bids (dict): dictionary that holds control over the final bids for each market and each hour
        chosen_assets (list): list of the chosen assets for each market and each hour

    Returns:
        bool: True if the bids are feasible, False if not
    """
    for episode in bids.keys():
        for hour in bids[episode]["Hour"].unique():
            for market in bids[episode]["Market"].loc[bids[episode]["Hour"] == hour]:
                #if not np.isnan(bids[episode][market].loc[hour]) and bids[episode][market].loc[hour] > 0:
                if "up" in market or "FCR_D" in market:
                    up_flex_vol = sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] for asset in chosen_assets[episode][(market, hour)][0]])
                    if up_flex_vol < 1:
                        print(f"up_flex_volume for {market} at {hour} in episode {episode} is only {up_flex_vol}")
                        return False
                    else:
                        continue
                elif "down" in market:
                    down_flex_vol = sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] for asset in chosen_assets[episode][(market, hour)][0]])
                    if down_flex_vol< 1:
                        print(f"down_flex_volume for {market} at {hour} in episode {episode} is only {down_flex_vol}")
                        return False
                    else:
                        continue
                else:
                    """print(f" episode : {type(episode)}")
                    print(f" hour : {hour}")
                    print(f" market : {market}")
                    print([asset.direction for asset in chosen_assets[episode][(market, hour)][0]])"""
                    #up_flex_vol = sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] for asset in chosen_assets[episode][(market, hour)]])
                    total_vol = 0
                    for asset in chosen_assets[episode][(market, hour)][0]:
                        if asset.direction == "both":
                            total_vol += min(asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0], asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0])
                        elif asset.direction == "up":
                            total_vol += asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0]
                        else:
                            total_vol += asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0]
                    if total_vol < 1:
                        print(f"total_flex_vol for {market} at {hour} in episode {episode} is only {total_vol}")
                        return False
        
    return True

In [347]:
bids[0].columns

Index(['Market', 'Hour', 'Asset Count', 'Total Flex Volume'], dtype='object')

In [348]:
feasibility_check_for_rl_model(bids, asset_bids)

True

In [92]:
bids[0].index[1]

Timestamp('2023-06-20 01:00:00+0200', tz='Europe/Oslo')

In [94]:
[asset.meter_id for asset in asset_bids[0][("aFRR up_NO5", bids[0].index[1])]]

[707057500029643616, 707057500029643623, 707057500029182443]

In [97]:
chosen_assets = asset_bids[0][("aFRR up_NO5", bids[0].index[1])]

In [98]:
total_flex_volume = sum(load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == bids[0].index[1]].values[0] for load in chosen_assets)


In [99]:
total_flex_volume

1.1752